In [1]:
import pandas as pd
import numpy as np
from numpy import vectorize as vec
import scipy as sp
import sklearn
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
#import seaborn as sns
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole 
from rdkit.Chem import Descriptors,PandasTools
from rdkit.ML.Descriptors import MoleculeDescriptors

In [2]:
labels = ["mu", "alpha", "homo", "lumo", "gap", "r2", "zpve", "u0", "u298", "h298", "g298", "cv"]

In [3]:
target_vec = pd.read_csv('datasets/qm9/qm9_shuffle.csv',usecols=labels)
# homo, u0
target_vec = target_vec.astype('float')
target_vec = target_vec.values.tolist()
target_vec = np.array(target_vec)
target_list = target_vec

In [4]:
print(target_list.shape)

(133802, 12)


In [5]:
train_target_list =  target_list[0:99968]
val_target_list = target_list[99968:99968 + 1000]

In [17]:
from mhg.nn.autoencoder import GrammarSeq2SeqVAE
from mhg.nn.dataset import HRGDataset, batch_padding
from mhg.smi import HGGen, hg_to_mol
from mhg.hrg import HyperedgeReplacementGrammar as HRG

import torch
from torch.utils.data import DataLoader

import os
import logging
import gzip
import pickle
import numpy as np
from copy import deepcopy

from rdkit import Chem

def get_dataloaders(hrg, prod_rule_seq_list, target_val_list=None, batch_size=None, shuffle=False):
    
    ''' return a dataloader for train/val/test

    Parameters
    ----------
    prod_rule_seq_list : List of lists
        each element corresponds to a sequence of production rules.
    train_params : dict
        self.Train_params

    Returns
    -------
    Dataloaders for train, val, test of autoencoders
        each batch contains two torch Tensors, each of which corresponds to input and output of autoencoder.
    '''
    # Parameters for training a variational autoencoder
    Train_params = {
        'model': 'GrammarSeq2SeqVAE', # Model name
        'model_params' : { # Parameter for the model
            'latent_dim': 50, # Dimension of the latent dimension
            'max_len': 12, # maximum length of input sequences (represented as sequences of production rules)
            'batch_size': 64, # batch size for training
            'padding_idx': -1, # integer used for padding
            'start_rule_embedding': False, # Embed the starting rule into the latent dimension explicitly
            'encoder_name': 'GRU', # Type of encoder
            'encoder_params': {'hidden_dim': 384, # hidden dim
                               'num_layers': 3, # the number of layers
                               'bidirectional': True, # use bidirectional one or not
                               'dropout': 0.1}, # dropout probability
            'decoder_name': 'GRU', # Type of decoder
            'decoder_params': {'hidden_dim': 384, # hidden dim
                               'num_layers': 3, # the number of layers
                               'dropout': 0.1}, # dropout probability
            'prod_rule_embed': ['Embedding',
                                'MolecularProdRuleEmbedding',
                                'MolecularProdRuleEmbeddingLastLayer',
                                'MolecularProdRuleEmbeddingUsingFeatures'][0], # Embedding method of a production rule. The simple embedding was the best, but each production rule could be embedded using GNN
            'prod_rule_embed_params': {'out_dim': 900, # embedding dimension
                                       'layer2layer_activation': 'relu', # not used for `Embedding`
                                       'layer2out_activation': 'softmax', # not used for `Embedding`
                                       'num_layers': 4}, # not used for `Embedding`
            'criterion_func': ['VAELoss', 'GrammarVAELoss'][1], # Loss function
            'criterion_func_kwargs': {'beta': 0.01}}, # Parameters for the loss function. `beta` specifies beta-VAE.
        'sgd': 'Adam', # SGD algorithm
        'sgd_params': {'lr': 5e-4 # learning rate of SGD
        },
        #'seed_list': [141, 123, 425, 552, 1004, 50243], # seeds used for restarting training
        'seed_list': [128], # seeds used for restarting training
        'inversed_input': True, # the input sequence is in the reversed order or not.
        'num_train':99968, # the number of training examples
        'num_val': 1024, # the number of validation examples
        'num_test': 28928, # the number of test examples
        'num_early_stop': 220011, # the number of examples used to find better initializations (=seed)
        'num_epochs': 10 # the number of training epochs
    }
    
    train_params = Train_params
    
    if batch_size is None:
        batch_size = train_params['model_params']['batch_size']
    prod_rule_seq_list_train = prod_rule_seq_list[: train_params['num_train']]
    prod_rule_seq_list_val = prod_rule_seq_list[train_params['num_train']
                                                : train_params['num_train'] + train_params['num_val']]
    prod_rule_seq_list_test = prod_rule_seq_list[train_params['num_train'] + train_params['num_val']
                                                 : train_params['num_train']
                                                 + train_params['num_val']
                                                 + train_params['num_test']]
    if target_val_list is None:
        target_val_list_train = None
        target_val_list_val = None
        target_val_list_test = None
    else:
        target_val_list_train = target_val_list[: train_params['num_train']]
        target_val_list_val = target_val_list[train_params['num_train']
                                              : train_params['num_train'] + train_params['num_val']]
        target_val_list_test = target_val_list[train_params['num_train'] + train_params['num_val']
                                               : train_params['num_train']
                                               + train_params['num_val']
                                               + train_params['num_test']]
    hrg_dataset_train = HRGDataset(hrg,
                                   prod_rule_seq_list_train,
                                   train_params['model_params']['max_len'],
                                   target_val_list=target_val_list_train,
                                   inversed_input=train_params['inversed_input'])
    hrg_dataloader_train = DataLoader(dataset=hrg_dataset_train,
                                      batch_size=batch_size,
                                      shuffle=shuffle, drop_last=False)
    if train_params['num_val'] != 0:
        hrg_dataset_val = HRGDataset(hrg,
                                     prod_rule_seq_list_val,
                                     train_params['model_params']['max_len'],
                                     target_val_list=target_val_list_val,
                                     inversed_input=train_params['inversed_input'])
        hrg_dataloader_label = DataLoader(dataset=hrg_dataset_val,
                                        batch_size=batch_size,
                                        shuffle=shuffle, drop_last=False)
    else:
        hrg_dataset_val = None
        hrg_dataloader_val = None
    if train_params['num_test'] != 0 :
        hrg_dataset_test = HRGDataset(hrg,
                                      prod_rule_seq_list_test,
                                      train_params['model_params']['max_len'],
                                      target_val_list=target_val_list_test,
                                      inversed_input=train_params['inversed_input'])
        hrg_dataloader_test = DataLoader(dataset=hrg_dataset_test,
                                         batch_size=batch_size,
                                         shuffle=shuffle, drop_last=False)
    else:
        hrg_dataset_test = None
        hrg_dataloader_test = None
    return hrg_dataloader_train, hrg_dataloader_label, hrg_dataloader_test, train_params


def load_output():
    input_dir_path = "OUTPUT/data_prep_for_qm9"
    with gzip.open(os.path.join(input_dir_path, 'mhg_prod_rules.pklz'), "rb") as f:
            hrg, prod_rule_seq_list = pickle.load(f)
    return hrg, prod_rule_seq_list

print("Start load input data and model")

hrg, prod_rule_seq_list = load_output()

hrg_dataloader_train, hrg_dataloader_val, hrg_dataloader_test, Train_params \
    = get_dataloaders(hrg, prod_rule_seq_list, target_val_list = target_list)

model_params = deepcopy(dict(Train_params['model_params']))

mhg_vae = GrammarSeq2SeqVAE(
    hrg=hrg,**model_params, use_gpu=True)

########################################  paper model    ##############################################

#mhg_vae = torch.load("OUTPUT/train/Joint_QM9/u0/joint_train_qm9/mhg_vae_joint_qm9.pt")
#mhg_vae = torch.load("OUTPUT/train/Metric_DRL_QM9/u0/balanced-semiLogRatio_train_qm9_tcbeta/mhg_vae_metric_qm9_12_model.pt")

#mhg_vae = torch.load("OUTPUT/train/Joint_QM9/homo/joint_train_qm9/mhg_vae_joint_qm9_14_model.pt")
#mhg_vae = torch.load("OUTPUT/train/Metric_DRL_QM9/homo/balanced-semiLogRatio_train_qm9_tcbeta/mhg_vae_metric_qm9_14_model.pt")

#mhg_vae = torch.load("OUTPUT/train/Joint_QM9/cv/joint_train_qm9/linear/mhg_vae_joint_qm9_25_model.pt")
#mhg_vae = torch.load("OUTPUT/train/Metric_DRL_QM9/cv/balanced-semiLogRatio_train_qm9_tcbeta/mhg_vae_metric_qm9_17_model.pt")

#mhg_vae = torch.load("OUTPUT/train/Metric_DRL_QM9/cv/balanced-semiLogRatio_train_qm9_tcbeta/mhg_vae_metric_qm9_23_model.pt")
mhg_vae = torch.load("OUTPUT/train/semiLogRatio_train_qm9_tcbeta_VAT_LUMO/mhg_vae_metric_qm9_19_model.pt")

##################################################################################################

mhg_vae = mhg_vae.to('cuda:0')

print("load completed")

Start load input data and model
load completed


# Data load

In [18]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

out_mu_train = []

mhg_vae.eval()

for each_idx, each_batch in enumerate(hrg_dataloader_train):
        
    print(each_idx)
        
    if len(each_batch[0]) < Train_params['model_params']['batch_size']:
                        each_batch[0] = torch.cat([each_batch[0],
                                                    Train_params['model_params']['padding_idx'] * torch.ones((Train_params['model_params']['batch_size'] - len(each_batch[0]),
                                                                                len(each_batch[0][0])), dtype=torch.int64)], dim=0)
                        each_batch[1] = torch.cat([each_batch[1],
                                                    Train_params['model_params']['padding_idx'] * torch.ones((Train_params['model_params']['batch_size'] - len(each_batch[1]),
                                                                                len(each_batch[1][0])), dtype=torch.int64)], dim=0)
                        each_batch[2] = torch.cat([each_batch[2],
                                                    torch.zeros((Train_params['model_params']['batch_size'] - len(each_batch[2])))], dim=0)

    if type(each_batch) == list:
            in_batch, out_batch, tgt_batch = each_batch
            in_batch = torch.LongTensor(np.mod(in_batch, mhg_vae.vocab_size))
            out_batch = torch.LongTensor(np.mod(out_batch, mhg_vae.vocab_size))
            tgt_batch = torch.FloatTensor(tgt_batch)
            mhg_vae.init_hidden()

            in_batch = in_batch.to("cuda:0")
            out_batch = out_batch.to("cuda:0")
            tgt_batch = tgt_batch.to("cuda:0")
        
            #encode
            mu, logvar = mhg_vae.encode(in_batch)
            z = mhg_vae.reparameterize(mu, logvar)
            
            mu = mu.to("cpu")
            tgt_batch  = tgt_batch.to("cpu")
            
            mu, tgt_batch = mu.data.numpy(), tgt_batch.data.numpy()
    
    out_mu_train.append(mu)
    
out_mu_train = np.array(out_mu_train)

out_mu_train = out_mu_train.reshape(99968, 50)
print(out_mu_train.shape)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [19]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

out_mu_val = []

mhg_vae.eval()

for each_idx, each_batch in enumerate(hrg_dataloader_val):
        
    print(each_idx)

    if len(each_batch[0]) < Train_params['model_params']['batch_size']:
                        each_batch[0] = torch.cat([each_batch[0],
                                                    Train_params['model_params']['padding_idx'] * torch.ones((Train_params['model_params']['batch_size'] - len(each_batch[0]),
                                                                                len(each_batch[0][0])), dtype=torch.int64)], dim=0)
                        each_batch[1] = torch.cat([each_batch[1],
                                                    Train_params['model_params']['padding_idx'] * torch.ones((Train_params['model_params']['batch_size'] - len(each_batch[1]),
                                                                                len(each_batch[1][0])), dtype=torch.int64)], dim=0)
                        each_batch[2] = torch.cat([each_batch[2],
                                                    torch.zeros((Train_params['model_params']['batch_size'] - len(each_batch[2])))], dim=0)

    if type(each_batch) == list:
            in_batch, out_batch, tgt_batch = each_batch
            in_batch = torch.LongTensor(np.mod(in_batch, mhg_vae.vocab_size))
            out_batch = torch.LongTensor(np.mod(out_batch, mhg_vae.vocab_size))
            tgt_batch = torch.FloatTensor(tgt_batch)
            mhg_vae.init_hidden()

            in_batch = in_batch.to("cuda:0")
            out_batch = out_batch.to("cuda:0")
            tgt_batch = tgt_batch.to("cuda:0")
        
            #encode
            mu, logvar = mhg_vae.encode(in_batch)
            z = mhg_vae.reparameterize(mu, logvar)
            
            mu = mu.to("cpu")
            tgt_batch  = tgt_batch.to("cpu")
            
            mu, tgt_batch = mu.data.numpy(), tgt_batch.data.numpy()
    
    out_mu_val.append(mu)

out_mu_val = np.array(out_mu_val)

print(out_mu_val.shape)

out_mu_val = out_mu_val.reshape(1024, 50)

print(out_mu_val.shape)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
(16, 64, 50)
(1024, 50)


# Linear regression with neighbor vector

## Sampling the physical property values and latent vectors of neighboring points

In [20]:
#計算コストがえげつないので、1000サンプルに限定する
anchor_tensor = out_mu_val[0:1000]
compare_tensor = out_mu_train

anchor_label = val_target_list[0:1000]
compare_label = train_target_list

In [21]:
import numpy as np

tensor_dis = np.zeros((len(anchor_tensor), len(compare_tensor)))

for anc_id, anc_vec in enumerate(anchor_tensor):
    
    print(anc_id)
    
    for comp_id, comp_vec in enumerate(compare_tensor):
        tens_dif = np.linalg.norm(comp_vec - anc_vec)
        tensor_dis[anc_id][comp_id] = tens_dif

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


KeyboardInterrupt: 

In [ ]:
neighbor_tensor_vec = np.zeros((len(anchor_tensor), 10, 50))
neighbor_label_vec = np.zeros((len(anchor_label), 10, 12))

for anc_id, comp_dis in enumerate(tensor_dis):
    print(anc_id)
    anc_comp_sorted_ids = comp_dis.argsort()
    for id, comp_id in enumerate(anc_comp_sorted_ids): #idが近傍の順番　comp_idがソートする前の近傍のインデックス
        
        if id == 10:
            break
      
        neighbor_tensor_vec[anc_id][id] = compare_tensor[comp_id]
        neighbor_label_vec[anc_id][id] = compare_label[comp_id]

# Save neighbor data

## u0

In [75]:
## center label

In [53]:
np.save("neighbor_regression_numpy/u0/joint_center_label_vec.npy", anchor_label)
np.save("neighbor_regression_numpy/u0/joint_center_tensor_vec.npy", anchor_tensor)

In [34]:
np.save("neighbor_regression_numpy/u0/metric_center_label_vec.npy", anchor_label)
np.save("neighbor_regression_numpy/u0/metric_center_tensor_vec.npy", anchor_tensor)

In [ ]:
# neighbor label

In [54]:
np.save("neighbor_regression_numpy/u0/joint_neighbor_vec.npy", neighbor_tensor_vec)
np.save("neighbor_regression_numpy/u0/joint_neighbor_label.npy", neighbor_label_vec)

In [35]:
np.save("neighbor_regression_numpy/u0/metric_neighbor_vec.npy", neighbor_tensor_vec)
np.save("neighbor_regression_numpy/u0/metric_neighbor_label.npy", neighbor_label_vec)

# homo

In [ ]:
#center

In [136]:
np.save("neighbor_regression_numpy/homo/joint_center_label_vec.npy", anchor_label)
np.save("neighbor_regression_numpy/homo/joint_center_tensor_vec.npy", anchor_tensor)

In [144]:
np.save("neighbor_regression_numpy/homo/metric_center_label_vec.npy", anchor_label)
np.save("neighbor_regression_numpy/homo/metric_center_tensor_vec.npy", anchor_tensor)

In [11]:
#neighbor

In [137]:
np.save("neighbor_regression_numpy/homo/joint_neighbor_vec.npy", neighbor_tensor_vec)
np.save("neighbor_regression_numpy/homo/joint_neighbor_label.npy", neighbor_label_vec)

In [145]:
np.save("neighbor_regression_numpy/homo/metric_neighbor_vec.npy", neighbor_tensor_vec)
np.save("neighbor_regression_numpy/homo/metric_neighbor_label.npy", neighbor_label_vec)

# cv

In [ ]:
# center

In [182]:
np.save("neighbor_regression_numpy/cv/joint_center_label_vec.npy", anchor_label)
np.save("neighbor_regression_numpy/cv/joint_center_tensor_vec.npy", anchor_tensor)

In [40]:
np.save("neighbor_regression_numpy/cv/metric_center_label_vec.npy", anchor_label)
np.save("neighbor_regression_numpy/cv/metric_center_tensor_vec.npy", anchor_tensor)

In [ ]:
#neighbor

In [183]:
np.save("neighbor_regression_numpy/cv/joint_neighbor_vec.npy", neighbor_tensor_vec)
np.save("neighbor_regression_numpy/cv/joint_neighbor_label.npy", neighbor_label_vec)

In [41]:
np.save("neighbor_regression_numpy/cv/metric_neighbor_vec.npy", neighbor_tensor_vec)
np.save("neighbor_regression_numpy/cv/metric_neighbor_label.npy", neighbor_label_vec)

## u0

In [14]:
#center 

In [16]:
center_label_vec = np.load("neighbor_regression_numpy/u0/joint_center_label_vec.npy")
center_tensor_vec = np.load("neighbor_regression_numpy/u0/joint_center_tensor_vec.npy")

In [36]:
center_label_vec = np.load("neighbor_regression_numpy/u0/metric_center_label_vec.npy")
center_tensor_vec = np.load("neighbor_regression_numpy/u0/metric_center_tensor_vec.npy")

In [ ]:
# neighbor

In [17]:
neighbor_tensor_vec = np.load("neighbor_regression_numpy/u0/joint_neighbor_vec.npy")
neighbor_label_vec = np.load("neighbor_regression_numpy/u0/joint_neighbor_label.npy")

In [37]:
neighbor_tensor_vec = np.load("neighbor_regression_numpy/u0/metric_neighbor_vec.npy")
neighbor_label_vec = np.load("neighbor_regression_numpy/u0/metric_neighbor_label.npy")

## homo

In [ ]:
# center

In [83]:
center_label_vec = np.load("neighbor_regression_numpy/homo/joint_center_label_vec.npy")
center_tensor_vec = np.load("neighbor_regression_numpy/homo/joint_center_tensor_vec.npy")

In [91]:
center_label_vec = np.load("neighbor_regression_numpy/homo/metric_center_label_vec.npy")
center_tensor_vec = np.load("neighbor_regression_numpy/homo/metric_center_tensor_vec.npy")

In [ ]:
# neighbor

In [84]:
neighbor_tensor_vec = np.load("neighbor_regression_numpy/homo/joint_neighbor_vec.npy")
neighbor_label_vec = np.load("neighbor_regression_numpy/homo/joint_neighbor_label.npy")

In [92]:
neighbor_tensor_vec = np.load("neighbor_regression_numpy/homo/metric_neighbor_vec.npy")
neighbor_label_vec = np.load("neighbor_regression_numpy/homo/metric_neighbor_label.npy")

## cv

In [ ]:
#center

In [44]:
center_label_vec = np.load("neighbor_regression_numpy/cv/joint_center_label_vec.npy")
center_tensor_vec = np.load("neighbor_regression_numpy/cv/joint_center_tensor_vec.npy")

In [42]:
center_label_vec = np.load("neighbor_regression_numpy/cv/metric_center_label_vec.npy")
center_tensor_vec = np.load("neighbor_regression_numpy/cv/metric_center_tensor_vec.npy")

In [ ]:
#neighbor

In [45]:
neighbor_tensor_vec = np.load("neighbor_regression_numpy/cv/joint_neighbor_vec.npy")
neighbor_label_vec = np.load("neighbor_regression_numpy/cv/joint_neighbor_label.npy")

In [43]:
neighbor_tensor_vec = np.load("neighbor_regression_numpy/cv/metric_neighbor_vec.npy")
neighbor_label_vec = np.load("neighbor_regression_numpy/cv/metric_neighbor_label.npy")

# Leave one (center) out - Linear regression -

In [11]:
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from scipy.stats import pearsonr

joint_mae = []
joint_r2 = []
joint_p =[]

metric_mae = []
metric_r2 = []
metric_p = []

pred_labels_tensor = []
truth_labels_tensor = []


for n, desc in enumerate(labels):
    
    print(desc)
    
    sum_error = 0

    pred_labels = []
    truth_labels = []

    neighbor_label = neighbor_label_vec[:,:,n:n+1]
    center_label = center_label_vec[:,n:n+1]

    for latent_vec, label_vec, c_vec, c_label in zip(neighbor_tensor_vec, neighbor_label, center_tensor_vec, center_label):

        center_vec = c_vec.reshape(-1,50)
        latent_vec = latent_vec[0:10]
    
        center_label = c_label
        label_vec = label_vec[0:10]
    
        lr = LinearRegression()
        lr.fit(latent_vec, label_vec)
        y_pred = lr.predict(center_vec)
    
        pred_labels.append(y_pred[0])
        truth_labels.append(center_label)
    
        print("pred:",y_pred[0], "  groud truth:",center_label)
        
    pred_labels_tensor.append(pred_labels)
    truth_labels_tensor.append(truth_labels)
    
    correlation, pvalue = pearsonr(pred_labels, truth_labels)
    
    print("MAE: %.5f" % mean_absolute_error(pred_labels, truth_labels))
    print("MSE: %.5f" % mean_squared_error(pred_labels, truth_labels))
    print('R2: %.5f' % r2_score(pred_labels, truth_labels))
    
    joint_mae.append(mean_absolute_error(pred_labels, truth_labels))
    joint_r2.append(r2_score(pred_labels, truth_labels))
    
    metric_mae.append(mean_absolute_error(pred_labels, truth_labels))
    metric_r2.append(r2_score(pred_labels, truth_labels))

mu
pred: [2.48032511]   groud truth: [2.4438]
pred: [2.43916009]   groud truth: [3.311]
pred: [1.06869643]   groud truth: [1.4136]
pred: [2.19308702]   groud truth: [1.4859]
pred: [3.19995967]   groud truth: [3.8287]
pred: [2.75417337]   groud truth: [2.1968]
pred: [2.33524642]   groud truth: [2.0996]
pred: [-1.07255944]   groud truth: [1.8511]
pred: [2.70482764]   groud truth: [1.747]
pred: [1.92825457]   groud truth: [1.8312]
pred: [1.72922423]   groud truth: [1.3219]
pred: [1.84785116]   groud truth: [2.1437]
pred: [2.72682338]   groud truth: [0.9789]
pred: [0.50742194]   groud truth: [0.2072]
pred: [0.41728876]   groud truth: [2.1384]
pred: [2.19500152]   groud truth: [3.5345]
pred: [1.57581193]   groud truth: [1.7719]
pred: [5.52737461]   groud truth: [4.7536]
pred: [4.86370466]   groud truth: [5.2353]
pred: [2.6514272]   groud truth: [2.8808]
pred: [1.85125646]   groud truth: [1.6092]
pred: [3.10152748]   groud truth: [3.3631]
pred: [1.31089515]   groud truth: [1.8586]
pred: [1.5

pred: [-1.70962719]   groud truth: [3.5623]
pred: [2.61673534]   groud truth: [2.4164]
pred: [1.45672861]   groud truth: [1.8563]
pred: [2.58525836]   groud truth: [2.5511]
pred: [1.84076434]   groud truth: [0.9957]
pred: [1.92243669]   groud truth: [1.23]
pred: [0.81059914]   groud truth: [1.29]
pred: [4.18314506]   groud truth: [4.0144]
pred: [1.95967245]   groud truth: [1.2663]
pred: [2.85648565]   groud truth: [2.6536]
pred: [3.8816339]   groud truth: [4.0367]
pred: [3.30855454]   groud truth: [3.6724]
pred: [1.56707218]   groud truth: [1.1706]
pred: [2.769567]   groud truth: [2.7941]
pred: [1.01843405]   groud truth: [1.1923]
pred: [1.73653676]   groud truth: [1.7944]
pred: [2.85637895]   groud truth: [2.8618]
pred: [2.12461364]   groud truth: [0.7654]
pred: [1.24829086]   groud truth: [2.3716]
pred: [1.17132258]   groud truth: [1.7575]
pred: [1.93554586]   groud truth: [0.6149]
pred: [1.05698418]   groud truth: [1.3726]
pred: [2.12599959]   groud truth: [1.174]
pred: [2.697734]  

pred: [3.05669639]   groud truth: [1.5253]
pred: [1.59315638]   groud truth: [1.212]
pred: [1.84077158]   groud truth: [1.6413]
pred: [0.28189545]   groud truth: [2.583]
pred: [2.81802425]   groud truth: [3.6292]
pred: [2.4352355]   groud truth: [1.9128]
pred: [4.51869291]   groud truth: [3.0208]
pred: [1.62747822]   groud truth: [1.9086]
pred: [2.24438044]   groud truth: [3.3955]
pred: [2.57328758]   groud truth: [3.5458]
pred: [0.62293155]   groud truth: [0.1145]
pred: [3.26011122]   groud truth: [2.0327]
pred: [3.80673277]   groud truth: [3.7801]
pred: [0.67456479]   groud truth: [1.1013]
pred: [3.75571899]   groud truth: [4.2202]
pred: [1.48784461]   groud truth: [2.2865]
pred: [2.33424326]   groud truth: [0.6542]
pred: [3.3887364]   groud truth: [5.619]
pred: [1.30489557]   groud truth: [1.0819]
pred: [4.77982329]   groud truth: [6.8283]
pred: [2.90565893]   groud truth: [4.0445]
pred: [2.08036364]   groud truth: [1.1507]
pred: [2.32549158]   groud truth: [3.2895]
pred: [3.0084061

pred: [79.27871544]   groud truth: [79.13]
pred: [71.90346249]   groud truth: [71.86]
pred: [80.4274371]   groud truth: [85.73]
pred: [71.01863221]   groud truth: [69.98]
pred: [80.24772523]   groud truth: [80.74]
pred: [75.51777657]   groud truth: [76.59]
pred: [85.7689623]   groud truth: [85.18]
pred: [70.87477833]   groud truth: [70.69]
pred: [76.62161994]   groud truth: [76.72]
pred: [75.55468427]   groud truth: [78.6]
pred: [72.94489269]   groud truth: [73.74]
pred: [66.96463424]   groud truth: [69.01]
pred: [75.25287288]   groud truth: [76.07]
pred: [85.20633387]   groud truth: [85.73]
pred: [73.80273869]   groud truth: [74.06]
pred: [79.63821179]   groud truth: [78.42]
pred: [61.78424203]   groud truth: [62.05]
pred: [71.51444579]   groud truth: [70.86]
pred: [83.07029827]   groud truth: [86.58]
pred: [81.42075599]   groud truth: [82.41]
pred: [56.92281468]   groud truth: [55.42]
pred: [82.9713286]   groud truth: [85.55]
pred: [75.22154073]   groud truth: [77.18]
pred: [62.69643

pred: [84.74707385]   groud truth: [88.29]
pred: [73.73042197]   groud truth: [74.75]
pred: [81.4671971]   groud truth: [81.94]
pred: [57.70376173]   groud truth: [59.61]
pred: [75.69524253]   groud truth: [75.29]
pred: [79.58301356]   groud truth: [78.72]
pred: [66.10553549]   groud truth: [66.03]
pred: [81.17718441]   groud truth: [78.61]
pred: [68.00267936]   groud truth: [66.89]
pred: [76.61408409]   groud truth: [77.07]
pred: [65.79290397]   groud truth: [65.75]
pred: [75.12418516]   groud truth: [75.24]
pred: [74.03714005]   groud truth: [74.39]
pred: [80.94762698]   groud truth: [81.16]
pred: [82.05767138]   groud truth: [83.4]
pred: [74.32755734]   groud truth: [74.6]
pred: [73.03279377]   groud truth: [75.37]
pred: [72.42777896]   groud truth: [71.83]
pred: [84.66611881]   groud truth: [85.43]
pred: [73.88472109]   groud truth: [73.52]
pred: [73.49446154]   groud truth: [73.7]
pred: [80.9805608]   groud truth: [79.25]
pred: [75.48565508]   groud truth: [73.28]
pred: [77.411514

...

pred: [-478.88015698]   groud truth: [-476.254101]
pred: [-359.26187429]   groud truth: [-362.588926]
pred: [-424.88408626]   groud truth: [-403.152962]
pred: [-412.08276485]   groud truth: [-399.781997]
pred: [-307.00950258]   groud truth: [-307.579988]
pred: [-468.22597299]   groud truth: [-473.837244]
pred: [-485.56440878]   groud truth: [-486.063919]
pred: [-430.28101347]   groud truth: [-420.502365]
pred: [-395.33485605]   groud truth: [-387.109419]
pred: [-408.42206864]   groud truth: [-382.07464]
pred: [-370.54779841]   groud truth: [-423.076393]
pred: [-459.06167277]   groud truth: [-476.272067]
pred: [-400.2840143]   groud truth: [-400.779923]
pred: [-403.77287349]   groud truth: [-400.851716]
pred: [-421.80012649]   groud truth: [-423.030345]
pred: [-366.8866988]   groud truth: [-361.455545]
pred: [-488.71808528]   groud truth: [-466.273336]
pred: [-427.06141804]   groud truth: [-423.090739]
pred: [-441.37330875]   groud truth: [-441.486781]
pred: [-416.88739474]   groud trut

pred: [-439.42905955]   groud truth: [-439.101609]
pred: [-403.40076541]   groud truth: [-404.36479]
pred: [-351.68726964]   groud truth: [-347.831108]
pred: [-389.71555204]   groud truth: [-403.175204]
pred: [-455.54278894]   groud truth: [-456.400208]
pred: [-420.68410053]   groud truth: [-423.068992]
pred: [-431.02702536]   groud truth: [-423.079394]
pred: [-543.18432974]   groud truth: [-566.071016]
pred: [-395.93136733]   groud truth: [-387.097538]
pred: [-386.27509123]   groud truth: [-387.077648]
pred: [-436.69318937]   groud truth: [-439.165364]
pred: [-403.68034507]   groud truth: [-383.357457]
pred: [-413.41025301]   groud truth: [-403.179908]
pred: [-420.15604216]   groud truth: [-418.009058]
pred: [-405.68232349]   groud truth: [-400.723299]
pred: [-425.30411408]   groud truth: [-425.454883]
pred: [-467.3410343]   groud truth: [-475.071709]
pred: [-393.84924943]   groud truth: [-388.346611]
pred: [-403.53921444]   groud truth: [-422.085057]
pred: [-386.64567628]   groud tru

pred: [25.81914118]   groud truth: [26.067]
pred: [32.39047202]   groud truth: [32.332]
pred: [26.40387388]   groud truth: [27.078]
pred: [31.45314537]   groud truth: [31.109]
pred: [34.49170491]   groud truth: [34.256]
pred: [26.7478118]   groud truth: [25.962]
pred: [33.29662477]   groud truth: [33.903]
pred: [34.0029497]   groud truth: [34.138]
pred: [28.9320758]   groud truth: [28.996]
pred: [31.59182951]   groud truth: [31.688]
pred: [36.26430682]   groud truth: [36.379]
pred: [29.64453431]   groud truth: [29.588]
pred: [26.65631901]   groud truth: [27.547]
pred: [31.68069981]   groud truth: [31.761]
pred: [30.32294016]   groud truth: [29.981]
pred: [40.97233286]   groud truth: [41.009]
pred: [27.92667046]   groud truth: [27.995]
pred: [28.22719943]   groud truth: [27.472]
pred: [27.00684102]   groud truth: [26.927]
pred: [27.95052904]   groud truth: [27.499]
pred: [32.80259751]   groud truth: [33.005]
pred: [36.42576]   groud truth: [36.674]
pred: [35.13673359]   groud truth: [33

pred: [30.0692675]   groud truth: [30.817]
pred: [32.79260629]   groud truth: [33.432]
pred: [26.02728545]   groud truth: [26.199]
pred: [30.81974954]   groud truth: [30.546]
pred: [29.30833897]   groud truth: [29.333]
pred: [36.0951771]   groud truth: [36.575]
pred: [33.68857556]   groud truth: [33.28]
pred: [25.05323277]   groud truth: [25.167]
pred: [33.21369777]   groud truth: [33.771]
pred: [33.94516907]   groud truth: [33.677]
pred: [35.1336638]   groud truth: [32.822]
pred: [29.3564135]   groud truth: [29.572]
pred: [25.46599929]   groud truth: [26.925]
pred: [31.68736167]   groud truth: [34.117]
pred: [34.35899682]   groud truth: [33.679]
pred: [25.59010811]   groud truth: [25.642]
pred: [38.44038304]   groud truth: [39.428]
pred: [31.13828309]   groud truth: [31.041]
pred: [34.48072979]   groud truth: [34.069]
pred: [28.49685259]   groud truth: [28.696]
pred: [25.29070112]   groud truth: [25.793]
pred: [33.87279822]   groud truth: [33.06]
pred: [34.58418449]   groud truth: [34

pred: [29.6373959]   groud truth: [30.029]
pred: [27.48359206]   groud truth: [27.442]
pred: [28.29855403]   groud truth: [28.442]
pred: [29.88426076]   groud truth: [30.094]
pred: [30.47341265]   groud truth: [31.007]
pred: [31.70823238]   groud truth: [31.494]
pred: [33.78268461]   groud truth: [33.891]
pred: [28.57284817]   groud truth: [27.409]
pred: [34.61723924]   groud truth: [34.784]
pred: [35.14655902]   groud truth: [35.374]
pred: [30.49078857]   groud truth: [30.577]
pred: [32.33367285]   groud truth: [32.565]
pred: [27.82367638]   groud truth: [26.997]
pred: [32.40369226]   groud truth: [32.25]
pred: [33.51860689]   groud truth: [33.097]
pred: [33.05018844]   groud truth: [33.011]
pred: [34.53663076]   groud truth: [33.914]
pred: [33.01429971]   groud truth: [33.663]
pred: [37.21110624]   groud truth: [36.181]
pred: [29.85458787]   groud truth: [30.469]
pred: [39.18419942]   groud truth: [38.402]
pred: [31.6267107]   groud truth: [31.081]
pred: [40.51112146]   groud truth: 

# for Supplement

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from scipy.stats import pearsonr

joint_mse = []
joint_r2 = []
joint_rmsd =[]

metric_mse = []
metric_r2 = []
metric_rmsd = []

pred_labels_tensor = []
truth_labels_tensor = []


for n, desc in enumerate(labels):
    
    print(desc)
    
    sum_error = 0

    pred_labels = []
    truth_labels = []

    neighbor_label = neighbor_label_vec[:,:,n:n+1]
    center_label = center_label_vec[:,n:n+1]

    for latent_vec, label_vec, c_vec, c_label in zip(neighbor_tensor_vec, neighbor_label, center_tensor_vec, center_label):

        center_vec = c_vec.reshape(-1,50)
        latent_vec = latent_vec[0:10]
    
        center_label = c_label
        label_vec = label_vec[0:10]
    
        lr = LinearRegression()
        lr.fit(latent_vec, label_vec)
        y_pred = lr.predict(center_vec)
    
        pred_labels.append(y_pred[0])
        truth_labels.append(center_label)
    
        print("pred:",y_pred[0], "  groud truth:",center_label)
        
    pred_labels_tensor.append(pred_labels)
    truth_labels_tensor.append(truth_labels)
    
    correlation, pvalue = pearsonr(pred_labels, truth_labels)
    
    print("MAE: %.5f" % mean_absolute_error(pred_labels, truth_labels))
    print("MSE: %.5f" % mean_squared_error(pred_labels, truth_labels))
    print("RMSE: %.5f" % np.sqrt(mean_squared_error(pred_labels, truth_labels)))
    print('R2: %.5f' % r2_score(pred_labels, truth_labels))
    
    joint_mse.append(mean_squared_error(pred_labels, truth_labels))
    joint_rmsd.append(np.sqrt(mean_squared_error(pred_labels, truth_labels)))
    joint_r2.append(r2_score(pred_labels, truth_labels))
    
    metric_mse.append(mean_squared_error(pred_labels, truth_labels))
    metric_rmsd.append(np.sqrt(mean_squared_error(pred_labels, truth_labels)))
    metric_r2.append(r2_score(pred_labels, truth_labels))

# Save results

# homo

In [152]:
np.save("neighbor_regression_results/homo/joint_neighbor_mae.npy", joint_mae)
np.save("neighbor_regression_results/homo/joint_neighbor_r2.npy", joint_r2)

In [32]:
np.save("neighbor_regression_results/homo/metric_neighbor_mae.npy", metric_mae)
np.save("neighbor_regression_results/homo/metric_neighbor_r2.npy", metric_r2)

## u0

In [58]:
np.save("neighbor_regression_results/u0/joint_neighbor_mae.npy", joint_mae)
np.save("neighbor_regression_results/u0/joint_neighbor_r2.npy", joint_r2)

In [39]:
np.save("neighbor_regression_results/u0/metric_neighbor_mae.npy", metric_mae)
np.save("neighbor_regression_results/u0/metric_neighbor_r2.npy", metric_r2)

# cv

In [190]:
np.save("neighbor_regression_results/cv/joint_neighbor_mae.npy", joint_mae)
np.save("neighbor_regression_results/cv/joint_neighbor_r2.npy", joint_r2)

In [205]:
np.save("neighbor_regression_results/cv/metric_neighbor_mae.npy", metric_mae)
np.save("neighbor_regression_results/cv/metric_neighbor_r2.npy", metric_r2)

# Load results

## homo

In [128]:
joint_mae = np.load("neighbor_regression_results/homo/joint_neighbor_mae.npy")
metric_mae = np.load("neighbor_regression_results/homo/metric_neighbor_mae.npy")

In [119]:
joint_r2 = np.load("neighbor_regression_results/homo/joint_neighbor_r2.npy")
metric_r2 = np.load("neighbor_regression_results/homo/metric_neighbor_r2.npy")

## cv

In [206]:
joint_mae = np.load("neighbor_regression_results/cv/joint_neighbor_mae.npy")
metric_mae = np.load("neighbor_regression_results/cv/metric_neighbor_mae.npy")

In [116]:
joint_r2 = np.load("neighbor_regression_results/cv/joint_neighbor_r2.npy")
metric_r2 = np.load("neighbor_regression_results/cv/metric_neighbor_r2.npy")

## u0

In [18]:
joint_mae = np.load("neighbor_regression_results/u0/joint_neighbor_mae.npy")
metric_mae = np.load("neighbor_regression_results/u0/metric_neighbor_mae.npy")

In [25]:
joint_mae = np.load("neighbor_regression_results/u0/joint_neighbor_mae.npy")
metric_mae = np.load("neighbor_regression_results/u0/metric_neighbor_mae_ave.npy")

# Table

In [ ]:
import pandas as pd

def highlight(s):
    is_min = s <= s.min()
    styles = is_min.map(lambda x: 'background-color: mediumslateblue; color: white' if x else '')
    return styles

physic_labels = ["mu", "alpha", "homo", "lumo", "gap", "r2", "zpve", "u0", "u298", "h298", "g298", "cv"]

model1 = ["joint"]
model2 = ["metric"]

df1 = pd.DataFrame(joint_mae, index=physic_labels, columns=model1)
df1_t = df1.T

df2 = pd.DataFrame(metric_mae, index=physic_labels,columns=model2)
df2_t = df2.T

df_concat = pd.concat([df1_t, df2_t])

df_concat2 = df_concat.style.apply(highlight, axis='index')

pd.options.display.float_format = '{:.3e}'.format

df_concat